In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Import Library

In [86]:
# importing libraries 
from sklearn.ensemble import VotingClassifier ,BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score 
from numpy import mean,std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from matplotlib import pyplot
from sklearn.datasets import load_wine,load_iris
from matplotlib.pyplot import figure
figure(num=2, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
import xgboost as xgb
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.linear_model import LinearRegression,BayesianRidge,ElasticNet,Lasso,SGDRegressor,Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,RobustScaler,StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA,KernelPCA
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor,VotingClassifier
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV,RandomizedSearchCV,StratifiedKFold,train_test_split
from sklearn.base import BaseEstimator,clone,TransformerMixin,RegressorMixin
from sklearn.svm import LinearSVR,SVR
from sklearn.ensemble import StackingClassifier
#import xgboost 
from xgboost import XGBRegressor, XGBClassifier
#Import Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew
from scipy.stats.stats import pearsonr
%matplotlib inline
seed = 1075
np.random.seed(seed)

<Figure size 1280x960 with 0 Axes>

# Load data

In [87]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [88]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [89]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [90]:
df.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [92]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [93]:
df[df['Cabin'].isnull() == False].value_counts('Survived')

Survived
1    136
0     68
Name: count, dtype: int64

In [94]:
df[df['Cabin'].isnull() == True].value_counts('Survived')

Survived
0    481
1    206
Name: count, dtype: int64

# Prepoccesing

In [95]:

def preprocessing(df):
    
    df = pd.get_dummies(df, columns=['Sex', 'Embarked'])
    df['Cabin'] = df['Cabin'].fillna('Unknown')
    df['Cabin'] = df['Cabin'].astype('category').cat.codes
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Fare'] = df["Fare"].fillna(df['Fare'].median())    
    return df

df_clean = preprocessing(df)


In [96]:
def scaleData(df):
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])  # Normalisasi semua kolom
    return df

In [97]:
X = df_clean.drop(columns=['Survived', 'PassengerId', 'Name', "Ticket"]) 
y = df['Survived']  


X_train, X_test, y_train, y_test = train_test_split(scaleData(X), y, test_size=0.2, random_state=42)


print(f"Ukuran train: {X_train.shape[0]} baris")
print(f"Ukuran test: {X_test.shape[0]} baris")


Ukuran train: 712 baris
Ukuran test: 179 baris


# Model


In [98]:
def train_and_evaluate_models(X, y, model):
 
    X_train, X_test, y_train, y_test = train_test_split(scaleData(X), y, test_size=0.2, random_state=42)

  

    # Melatih dan mengevaluasi setiap model
    results = {}
    estimator = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
#         report = classification_report(y_test, y_pred, output_dict=True)
        results[model_name] = {
            "accuracy": accuracy,
        }
        estimator[model_name] = model
    

    return results, estimator

In [99]:
  models = {
        "Random Forest": RandomForestClassifier(random_state=42),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "CatBoost": CatBoostClassifier(iterations=1000, learning_rate=0.1, verbose = False)
    }

In [100]:
accuracy, model = train_and_evaluate_models(X, y, models)

In [101]:
pd.DataFrame(accuracy).T

,accuracy
Random Forest,0.804469
XGBoost,0.782123
Gradient Boosting,0.832402
CatBoost,0.787709


# Max voting / hard voting

In [102]:
def train_and_evaluate_voting(X, y, type_voting, Models):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    voting_model = VotingClassifier(estimators=Models, voting= type_voting )

    voting_model.fit(X_train, y_train)


    y_pred = voting_model.predict(X_test)

    # Menghitung akurasi
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy, voting_model

In [103]:
rf_model = RandomForestClassifier(random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
gb_model = GradientBoostingClassifier(random_state=42)
cat_boost = CatBoostClassifier(iterations=100, learning_rate=0.1, verbose = False)

Estimator_tree = [('rf', rf_model),
        ('xgb', xgb_model),
        ('gb', gb_model),
        ('cat', cat_boost)]

In [104]:
accuracy_hard, model_hard = train_and_evaluate_voting(X, y, "hard", Estimator_tree ) 
model_hard

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importanc...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x78f87857f460>)])

In [105]:
print(f"Akurasi Hard Voting: {accuracy_hard:.2f}")

Akurasi Hard Voting: 0.83


In [106]:
lr_model = LogisticRegression()
svc_model = SVC()
dtc_model = DecisionTreeClassifier()

Estimator_classifiers = [
    ('lr', lr_model),
    ('svc', svc_model),
    ('dtc', dtc_model)
]


In [107]:
accuracy_hard_2, model_hard2 = train_and_evaluate_voting(X, y, "hard", Estimator_classifiers ) 
model_hard2

VotingClassifier(estimators=[('lr', LogisticRegression()), ('svc', SVC()),
                             ('dtc', DecisionTreeClassifier())])

In [108]:
print(f"Akurasi Hard Voting: {accuracy_hard_2:.2f}")

Akurasi Hard Voting: 0.83


In [109]:
accuracy_hard_3, model_hard_all = train_and_evaluate_voting(X, y, "hard", Estimator_classifiers + Estimator_tree ) 
model_hard_all

VotingClassifier(estimators=[('lr', LogisticRegression()), ('svc', SVC()),
                             ('dtc', DecisionTreeClassifier()),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eva...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x78f87857f460>)])

In [110]:
print(f"Akurasi Hard Voting: {accuracy_hard_3:.2f}")

Akurasi Hard Voting: 0.84


# Soft Voting

In [111]:
rf_model = RandomForestClassifier(random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
gb_model = GradientBoostingClassifier(random_state=42)
cat_boost = CatBoostClassifier(iterations=100, learning_rate=0.1, verbose = False)

Estimator_tree = [('rf', rf_model),
        ('xgb', xgb_model),
        ('gb', gb_model),
        ('cat', cat_boost)]

In [112]:
accuracy_soft, model_soft = train_and_evaluate_voting(X, y, "soft", Estimator_tree) 
model_soft

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importanc...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x78f87857fd90>)],
                 voting='soft')

In [113]:
print(f"Akurasi soft Voting: {accuracy_soft:.2f}")

Akurasi soft Voting: 0.82


In [114]:


lr_model = LogisticRegression()
svc_model = SVC(probability=True)
dtc_model = DecisionTreeClassifier()

Estimator_classifiers = [
    ('lr', lr_model),
    ('svc', svc_model),
    ('dtc', dtc_model)
]


In [115]:
accuracy_soft_2, model_hard2 = train_and_evaluate_voting(X, y, "soft", Estimator_classifiers ) 
model_hard2

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svc', SVC(probability=True)),
                             ('dtc', DecisionTreeClassifier())],
                 voting='soft')

In [116]:
print(f"Akurasi soft Voting: {accuracy_soft_2:.2f}")

Akurasi soft Voting: 0.85


# Stacking

In [117]:
estimators = [
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('tree', DecisionTreeClassifier())
]


meta_model = LogisticRegression(solver='lbfgs', max_iter=1000)


stacking_clf = StackingClassifier(estimators=estimators, final_estimator=meta_model)


stacking_clf.fit(X_train, y_train)


accuracy = stacking_clf.score(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 82.12%


# Submit 

In [118]:
submit_df = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
submit_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [119]:
submit = preprocessing(test)

In [120]:
submit[X_train.columns].isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [130]:
submit_df['Survived'] = model_hard.predict(scaleData(submit[X_train.columns]))

In [131]:
submit_df.to_csv('Submit_hard_voting_tree_mode_ensembel.csv', index = False)

In [132]:
model['XGBoost'].predict(scaleData(submit[X_train.columns]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,